In [ ]:
import csv
import pandas as pd
import numpy as np
import pprint
from IPython.display import display

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
tonnages_warstatus_raw = []
with open('./tonnages_warstatus.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:  
        tonnages_warstatus_raw.append(dict(row))
tonnages_warstatus_for_vega = []
for t in tonnages_warstatus_raw:
    annee = t["annee"]
    keys = t.keys()
    for key in keys:
        if key != "annee":
            tonnages_warstatus_for_vega.append({
                "annee": annee,
                "status": key,
                "part": t[key]
            })
            
tonnages_warstatus_levant_only_raw = []
with open('./tonnages_warstatus_levant_only.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:  
        tonnages_warstatus_levant_only_raw.append(dict(row))
tonnages_warstatus_levant_only_for_vega = []
for t in tonnages_warstatus_levant_only_raw:
    annee = t["annee"]
    keys = t.keys()
    for key in keys:
        if key != "annee":
            tonnages_warstatus_levant_only_for_vega.append({
                "annee": annee,
                "status": key,
                "part": t[key]
            })

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Tonnage par statut de guerre (sources patentes uniquement)",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar"
    },
    "data": {
        "values": tonnages_warstatus_for_vega
    },
    "encoding": {
        "x": {
            "field": "annee",
            "type": "nominal", 
            "title": "année",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "part",
            "aggregate": "mean",
            "type": "quantitative",
            "title": "part"
        },
        "color": {
            "field": "status",
            "type": "nominal",
            "title": "statut dans les guerres",
            "scale": {
                "domain": [
                    "en guerre ennemi de la France",
                    "France du Levant",
                    "France du Ponant",
                    "en guerre allié de la France",
                    "neutre"
                ],
                "range": [
                    "red",
                    "cornflowerblue",
                    "darkblue",
                    "darkseagreen",
                    "grey"
                ]
            }
        }
    }
})

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Tonnage par statut de guerre (sources patentes uniquement) (levant seulement)",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar"
    },
    "data": {
        "values": tonnages_warstatus_levant_only_for_vega
    },
    "encoding": {
        "x": {
            "field": "annee",
            "type": "nominal", 
            "title": "année",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "part",
            "aggregate": "mean",
            "type": "quantitative",
            "title": "part"
        },
        "color": {
            "field": "status",
            "type": "nominal",
            "title": "statut dans les guerres",
            "scale": {
                "domain": [
                    "en guerre ennemi de la France",
                    "France du Levant",
                    "France du Ponant",
                    "en guerre allié de la France",
                    "neutre"
                ],
                "range": [
                    "red",
                    "cornflowerblue",
                    "darkblue",
                    "darkseagreen",
                    "grey"
                ]
            }
        }
    }
})

In [ ]:
import requests
import csv

GUERRE_STATUS_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRxr78qK0zubFIDR7w38JqWOkqD-P6uqZi-pXChSLwYxesUGJwZnxCx-0sYIKKxwr3SJvb5P5HVGW1o/pub?gid=1380138397&single=true&output=csv'

war_status = []
with requests.Session() as s:
    download = s.get(GUERRE_STATUS_URL)
    decoded_content = download.content.decode('utf-8')
    reader = csv.DictReader(decoded_content.splitlines())
    for row in reader:
        war_status.append(row)

war_status_map = {}
for state in war_status:
    # war_status_map[state["state_navigo_1789"]] = state
    war_status_map[state["Pavillon"]] = state

In [ ]:
flows_to_Marseille = []
rank_Marseille = {}
with open('../../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['destination_function'] == "O" \
        and row["source_suite"] == "la Santé registre de patentes de Marseille":
           # and row['toponyme_fr'] == 'Marseille' \
            flows_to_Marseille.append(row)
            rank_Marseille[row["source_doc_id"]] = row["travel_rank"]

for row in flows_to_Marseille:
    doc_id = row["source_doc_id"]
    # travel = travels[doc_id]
    distance = row["distance_dep_dest_miles"]
    flag = row["ship_flag_standardized_fr"]
    year = row["indate_fixed"][:4]
    row["year"] = year
    if year == "1787":
        continue
    war_status = "inconnu"
    if flag == "français":
        war_status = "France"
    elif flag != "" and flag in war_status_map:
        war_status = war_status_map[flag][year]
    row["ship_war_status"] = war_status

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distance moyenne par statut de guerre (sources patentes uniquement)",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar"
    },
    "data": {
        "values": [f for f in flows_to_Marseille if int(f["distance_dep_dest_miles"] or 0) > 0]
    },
    "encoding": {
        "x": {
            "field": "year",
            "type": "nominal", 
            "title": "année",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "distance_dep_dest_miles",
            "aggregate": "mean",
            "type": "quantitative",
            "title": "distance"
        },
        "row": {
            "field": "ship_war_status"
        },
        "color": {
            "field": "ship_war_status",
            "type": "nominal",
            "title": "statut dans les guerres",
            "scale": {
                "domain": [
                    "en guerre ennemi de la France",
                    "France du Levant",
                    "France du Ponant",
                    "en guerre allié de la France",
                    "neutre"
                ],
                "range": [
                    "red",
                    "cornflowerblue",
                    "darkblue",
                    "darkseagreen",
                    "grey"
                ]
            }
        }
    }
})